<a href="https://colab.research.google.com/github/bencode/tfadv/blob/main/C1_W5_Lab_1_exploring_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.5.0


In [3]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [5]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [6]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [7]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [9]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [10]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [11]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [12]:
!rm -rf logs

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 23s 807ms/step - loss: 0.6779 - accuracy: 0.5560 - val_loss: 0.6749 - val_accuracy: 0.5073
Epoch 2/10
26/26 [==============================] - 21s 765ms/step - loss: 0.6425 - accuracy: 0.6302 - val_loss: 0.6269 - val_accuracy: 0.6683
Epoch 3/10
26/26 [==============================] - 22s 795ms/step - loss: 0.6100 - accuracy: 0.6667 - val_loss: 0.6434 - val_accuracy: 0.5512
Epoch 4/10
26/26 [==============================] - 22s 799ms/step - loss: 0.5671 - accuracy: 0.7287 - val_loss: 0.5080 - val_accuracy: 0.8878
Epoch 5/10
26/26 [==============================] - 22s 795ms/step - loss: 0.5247 - accuracy: 0.7555 - val_loss: 0.5710 - val_accuracy: 0.6488
Epoch 6/10
26/26 [==============================] - 22s 786ms/step - loss: 0.4880 - accuracy: 0.8005 - val_loss: 0.3983 - val_accuracy: 0.9220
Epoch 7/10
26/26 [==============================] - 21s 773ms/step - loss: 0.4132 - accuracy: 0.8370 - val_loss: 0.3350 - val_accuracy: 0.9171